In [9]:
from bs4 import BeautifulSoup as Soup
from IPython.display import HTML
import pandas as pd
import numpy as np
import re
import pprint
from IPython.display import display

In [10]:
fn = 'orig.procPublicationRequestDMSSPortal.oct-dec-2014.csv'
rows = pd.read_csv(fn, header=0, sep='|', encoding='latin-1')

In [11]:
cols = ['RequestID', 'StartDate', 'EndDate', 
        'AgencyCode', 'AgencyName', 'AgencyDivision', 
        'TypeOfNoticeCode', 'TypeOfNoticeDescription', 
        'CategoryCode', 'CategoryDescription', 'ShortTitle', 
        'SelectionMethodCode', 'SelectionMethodDescription', 
        'SectionID', 'SectionName', 'SpecialCaseReasonCode', 
        'SpecialCaseReasonDescription', 'PIN', 'DueDate', 
        'AddressToRequest', 'ContactName', 'ContactPhone', 
        'Email', 'AddressToSubmit', 'ContractAmount', 
        'RequestedBy', 'PhoneRequested', 'ConfirmationNumber', 
        'ContactFax', 'AdditionalDescription', 'OtherInfo', 
        'VendorName', 'VendorAddress', 'Printout']

In [14]:
cas = rows['AgencyName'] == "Citywide Administrative Services"
awards = rows['TypeOfNoticeDescription'] == "Award"
rows = rows[cas & awards][cols]
rows

,RequestID,StartDate,EndDate,AgencyCode,AgencyName,AgencyDivision,TypeOfNoticeCode,TypeOfNoticeDescription,CategoryCode,CategoryDescription,...,ContractAmount,RequestedBy,PhoneRequested,ConfirmationNumber,ContactFax,AdditionalDescription,OtherInfo,VendorName,VendorAddress,Printout
68,20140926001,2014-10-02 00:00:00,2014-10-02 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,2300000.00,someuser,5511234567,20140926001,NaN,NaN,Award made based on negotiated acquisition as ...,Lyngsoe Systems Inc.,"7470 New Technology Way, Suite P Frederick, M...",AUTOMATED SELF CHECK IN SYSTEM-QUEENS LIBRARY ...
69,20141020020,2014-10-27 00:00:00,2014-10-27 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,993785.00,someuser,5511234567,20141020020,NaN,NaN,NaN,PCS Pump and Process Inc.,"41 Plymouth Street, Fairfield, NY 07004",FAIRBANKS MORSE PUMPS - Competitive Sealed Bid...
70,20140925013,2014-10-02 00:00:00,2014-10-02 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,616000.00,someuser,5511234567,20140925013,NaN,NaN,NaN,"Henrich Equipment Co., Inc","42 Field Street, West Babylon, NY 11704","GRP: SPATCO EQUIPMENT CO., INC - Competitive S..."
71,20141020010,2014-10-27 00:00:00,2014-10-27 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,363566.94,someuser,5511234567,20141020010,NaN,NaN,NaN,Video Corporation of America,"7 Veronica Ave., PO BOX 5480, Somerset, NJ 088...","HD CONVERSION EQUIPMENT, AUDIO VISUAL (CITY HA..."
72,20140926009,2014-10-03 00:00:00,2014-10-03 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,1529364.40,someuser,5511234567,20140926009,NaN,NaN,NaN,Hydra-Shield Manufacturing Inc.,"8701 Carpenter Freeway, Suite 230, Dallas, TX ...","HYDRANT LOCKING DEVICES, (CUSTODIAN) BRAND SPE..."
73,20140924036,2014-10-01 00:00:00,2014-10-01 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,660352.00,someuser,5511234567,20140924036,NaN,NaN,NaN,Impact Recovery Systems INc.,"4955 Stout Drive, San Antonio, TX 78219","LANE SEPARATOR SYSTEMS, BRAND SPECIFIC (DOT) -..."
74,20140924038,2014-10-01 00:00:00,2014-10-01 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,36741.12,someuser,5511234567,20140924038,NaN,NaN,NaN,Elwood International Inc.,"89 Hudson Street, Copiague, NY 11726",MAYONNAISE-DOC RE AD - Competitive Sealed Bids...
75,20140924011,2014-10-01 00:00:00,2014-10-01 00:00:00,856,Citywide Administrative Services,NaN,2,Award,7,Services (other than human services),...,3000000.00,someuser,5511234567,20140924011,NaN,"Requirement contract for interior painting, pl...",NaN,INNOVATIVE CONSTRUCTION and MANAGEMENT CO. INC.,"42-02 Layton Street - Suite #202, Elmhurst, Ne...","PAINTING, PLASTER AND PATCHING WORK AT OCA FAC..."
76,20140924037,2014-10-01 00:00:00,2014-10-01 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,277920.00,someuser,5511234567,20140924037,NaN,NaN,NaN,Edsal Machine Products Inc.,"126 56th Street, Brooklyn, NY 11220","PIPE, BIKE RACK - Competitive Sealed Bids - PI..."
77,20141103013,2014-11-12 00:00:00,2014-11-12 00:00:00,856,Citywide Administrative Services,NaN,2,Award,1,Goods,...,1374750.00,someuser,5511234567,20141103013,NaN,NaN,NaN,Tourbillion Trailer Sales Inc,"401 Snake Hill Road, North Scituate, RI 02857","TRAILER, TWO HORSE, NYPD - Competitive Sealed ..."


In [13]:
from IPython.display import display
badcount = 0
for html in rows.AdditionalDescription.values:
    if not isinstance(html, str):
        badcount += 1
    else:
        display(HTML(html))
        try:
            parse_notice(Soup(html))
        except Exception:
            badcount += 1

In [ ]:
import json
def scrape(row):
    output = None
    try:
        output = parse_notice(Soup(row.AdditionalDescription))
    except Exception :
        output = {'error' : 'bad input: [{}]'.format(row.AdditionalDescription)}
    row['output'] = json.dumps(output)
    return row

In [ ]:
fix = rows.apply(scrape,1)

In [ ]:
for rec in fix[['output', 'AdditionalDescription']].values:
    output, text = rec
    if not isinstance(text, str):
        text = 'NAN'
    display(HTML(text))
    print('---')
    pprint.pprint(json.loads(output))
    display(HTML('<hr />'))
    print('\n\n')